In [110]:
import torch                                        # root package
from torch.utils.data import Dataset, DataLoader    # dataset representation and loading

In [111]:
# 🚀 STEP 1: Import Required Libraries
import os
import re
import json
import orjson
import pandas as pd

# 🚀 STEP 2: Check if JSON File Exists and is Not Empty
file_path = "data/climbs.json"

if not os.path.exists(file_path):
    print("🚨 Error: File not found!")
elif os.path.getsize(file_path) == 0:
    print("🚨 Error: The JSON file is empty!")
else:
    print(f"✅ JSON file found, size: {os.path.getsize(file_path)} bytes")

# 🚀 STEP 3: Read Raw JSON Content to Check for Issues
with open(file_path, "r") as file:
    raw_json = file.read()

# Print first 1000 characters to inspect JSON structure
print("\n🔍 First 1000 characters of JSON file:\n", raw_json[:1000])

# 🚀 STEP 4: Clean JSON (Remove Trailing Commas)
clean_json = re.sub(r",\s*([\]}])", r"\1", raw_json)

# Save cleaned JSON
cleaned_file_path = "data/climbs_cleaned.json"
with open(cleaned_file_path, "w") as file:
    file.write(clean_json)

print("✅ Cleaned JSON saved as:", cleaned_file_path)

# 🚀 STEP 5: Load JSON Safely with orjson
try:
    with open(cleaned_file_path, "rb") as file:
        climbs_data = orjson.loads(file.read())
    print("✅ Successfully loaded cleaned JSON!")
except orjson.JSONDecodeError as e:
    print("🚨 JSON Error:", e)
    climbs_data = None  # Set to None to avoid issues

# 🚀 STEP 6: Convert JSON into Pandas DataFrame
if climbs_data and isinstance(climbs_data, list) and "placements" in climbs_data[0]:
    df = pd.json_normalize(climbs_data, record_path="placements")
    print("✅ Data successfully converted to Pandas DataFrame!")
else:
    print("🚨 Error: JSON structure is not as expected!")

# 🚀 STEP 7: Display DataFrame in Jupyter Notebook
if "df" in locals():
    display(df.head())  # Jupyter's built-in display

# 🚀 STEP 8: Save DataFrame as CSV for ML Processing
if "df" in locals():
    df.to_csv("climbing_routes.csv", index=False)
    print("✅ Data saved as climbing_routes.csv")

# 🚀 STEP 9: Inspect the DataFrame Structure
if "df" in locals():
    print("\n🔍 DataFrame Info:\n")
    print(df.info())

    print("\n🔍 Sample Data:\n")
    print(df.head())


✅ JSON file found, size: 9785572 bytes

🔍 First 1000 characters of JSON file:
 [
    {"difficulty": 18.545, "placements": [{"x": 20, "y": 0, "type": "FEET-ONLY", "ledPosition": 15}, {"x": 8, "y": 3, "type": "FEET-ONLY", "ledPosition": 143}, {"x": 16, "y": 5, "type": "START", "ledPosition": 244}, {"x": 12, "y": 7, "type": "START", "ledPosition": 191}, {"x": 14, "y": 9, "type": "MIDDLE", "ledPosition": 203}, {"x": 22, "y": 13, "type": "MIDDLE", "ledPosition": 308}, {"x": 8, "y": 19, "type": "MIDDLE", "ledPosition": 131}, {"x": 16, "y": 19, "type": "MIDDLE", "ledPosition": 233}, {"x": 12, "y": 23, "type": "MIDDLE", "ledPosition": 179}, {"x": 8, "y": 29, "type": "MIDDLE", "ledPosition": 124}, {"x": 14, "y": 31, "type": "MIDDLE", "ledPosition": 219}, {"x": 14, "y": 35, "type": "FINISH", "ledPosition": 221}]},
    {"difficulty": 12.9173, "placements": [{"x": 14, "y": 3, "type": "FEET-ONLY", "ledPosition": 198}, {"x": 16, "y": 5, "type": "FEET-ONLY", "ledPosition": 244}, {"x": 8, "y": 7, "typ

,x,y,type,ledPosition
0,20,0,FEET-ONLY,15.0
1,8,3,FEET-ONLY,143.0
2,16,5,START,244.0
3,12,7,START,191.0
4,14,9,MIDDLE,203.0


✅ Data saved as climbing_routes.csv

🔍 DataFrame Info:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159508 entries, 0 to 159507
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   x            159508 non-null  int64  
 1   y            159508 non-null  int64  
 2   type         157868 non-null  object 
 3   ledPosition  157994 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 4.9+ MB
None

🔍 Sample Data:

    x  y       type  ledPosition
0  20  0  FEET-ONLY         15.0
1   8  3  FEET-ONLY        143.0
2  16  5      START        244.0
3  12  7      START        191.0
4  14  9     MIDDLE        203.0


In [112]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Encoding hold type to numerical values
type_encoding = {"START": 0, "MIDDLE": 1, "FINISH": 2, "FEET-ONLY": 3}
df["type"] = df["type"].map(type_encoding)

# Normalize x and y values
# scaler = MinMaxScaler()
# df[["x", "y"]] = scaler.fit_transform(df[["x", "y"]])

# Convert to NumPy array for training
data = df[["x", "y", "type"]].values

print("✅ Data processed for ML Training!")
print(data[:5])  # Display first 5 samples


✅ Data processed for ML Training!
[[20.  0.  3.]
 [ 8.  3.  3.]
 [16.  5.  0.]
 [12.  7.  0.]
 [14.  9.  1.]]


In [113]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch
from torch.utils.data import Dataset, DataLoader

# Load processed DataFrame
df = pd.DataFrame(climbs_data)

# Convert placements into a DataFrame
df = df.explode("placements").reset_index(drop=True)
df = pd.json_normalize(df["placements"])

# Encode hold types as numbers
type_encoding = {"START": 0, "MIDDLE": 1, "FINISH": 2, "FEET-ONLY": 3}
df["type"] = df["type"].map(type_encoding)

# Normalize x, y values
# # scaler = MinMaxScaler()
# df[["x", "y"]] = scaler.fit_transform(df[["x", "y"]])

# Convert DataFrame to NumPy array
data = df[["x", "y", "type"]].values  # Shape: (num_holds, 3)

print("✅ Data Ready for Training!")
print(data[:5])  # Show first 5 climbing holds


✅ Data Ready for Training!
[[20.  0.  3.]
 [ 8.  3.  3.]
 [16.  5.  0.]
 [12.  7.  0.]
 [14.  9.  1.]]


In [114]:
# Define sequence length (number of past holds to consider)
sequence_length = 5  

# Function to create sequences
def create_sequences(data, seq_length):
    X, Y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])  # Past `seq_length` holds
        Y.append(data[i+seq_length])  # Next hold to predict
    return np.array(X), np.array(Y)

# Generate sequences
X, Y = create_sequences(data, sequence_length)

# Convert to PyTorch tensors
X_train = torch.tensor(X, dtype=torch.float32)
Y_train = torch.tensor(Y, dtype=torch.float32)

print(f"✅ Training Data Shape: {X_train.shape}, Labels Shape: {Y_train.shape}")


✅ Training Data Shape: torch.Size([159503, 5, 3]), Labels Shape: torch.Size([159503, 3])


In [115]:
import numpy as np

print("🚀 Checking for NaNs in training data...")

if np.isnan(X_train.numpy()).any():
    print("🚨 NaN detected in X_train!")
if np.isnan(Y_train.numpy()).any():
    print("🚨 NaN detected in Y_train!")

print("✅ No NaNs found in training data!")


🚀 Checking for NaNs in training data...
🚨 NaN detected in X_train!
🚨 NaN detected in Y_train!
✅ No NaNs found in training data!


In [116]:
df.fillna(0, inplace=True)  # Replace NaNs with 0 before scaling
df.replace([np.inf, -np.inf], 0, inplace=True)  # Replace infinities too

# Normalize x, y
scaler = MinMaxScaler()
df[["x", "y"]] = scaler.fit_transform(df[["x", "y"]])

print("✅ NaNs removed before scaling!")


✅ NaNs removed before scaling!


In [117]:
# Convert X_train, Y_train to NumPy
X_train = np.nan_to_num(X_train.numpy())  # Replace NaNs with 0
Y_train = np.nan_to_num(Y_train.numpy())

# Convert back to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
Y_train = torch.tensor(Y_train, dtype=torch.float32)

print("✅ Training data cleaned (No NaNs)!")

print("🚀 Checking for NaNs in training data...")

if torch.isnan(X_train).any():
    print("🚨 NaN detected in X_train!")
if torch.isnan(Y_train).any():
    print("🚨 NaN detected in Y_train!")
    
print("✅ No NaNs found in training data!")


✅ Training data cleaned (No NaNs)!
🚀 Checking for NaNs in training data...
✅ No NaNs found in training data!


In [118]:
import torch.nn as nn

# Define LSTM Model
class ClimbRouteLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ClimbRouteLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        # Take the last output as the next hold prediction
        out = self.fc(lstm_out[:, -1, :])
        return out

# Model Parameters
input_size = 3  # (x, y, type)
hidden_size = 64
output_size = 3  # Predict (next x, next y, next type)

# Create Model
model = ClimbRouteLSTM(input_size, hidden_size, output_size)

print("✅ LSTM Model Ready!")


✅ LSTM Model Ready!


In [121]:
import torch.optim as optim

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
num_epochs = 500
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, Y_train)
    loss.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

print("🎉 Training Complete!")


Epoch 1/500, Loss: 165.7995
Epoch 11/500, Loss: 155.1292
Epoch 21/500, Loss: 145.6124
Epoch 31/500, Loss: 136.9871
Epoch 41/500, Loss: 128.7254
Epoch 51/500, Loss: 120.9006
Epoch 61/500, Loss: 114.2941
Epoch 71/500, Loss: 108.8412
Epoch 81/500, Loss: 104.1700
Epoch 91/500, Loss: 100.1325
Epoch 101/500, Loss: 96.6443
Epoch 111/500, Loss: 93.6334
Epoch 121/500, Loss: 91.0307
Epoch 131/500, Loss: 88.7785
Epoch 141/500, Loss: 86.8065
Epoch 151/500, Loss: 85.0789
Epoch 161/500, Loss: 83.6216
Epoch 171/500, Loss: 82.2893
Epoch 181/500, Loss: 80.9224
Epoch 191/500, Loss: 79.6825
Epoch 201/500, Loss: 78.4247
Epoch 211/500, Loss: 77.2799
Epoch 221/500, Loss: 76.1635
Epoch 231/500, Loss: 75.1901
Epoch 241/500, Loss: 74.2989
Epoch 251/500, Loss: 73.4501
Epoch 261/500, Loss: 72.5301
Epoch 271/500, Loss: 71.6974
Epoch 281/500, Loss: 70.9639
Epoch 291/500, Loss: 70.2993
Epoch 301/500, Loss: 69.6511
Epoch 311/500, Loss: 69.0624
Epoch 321/500, Loss: 68.5307
Epoch 331/500, Loss: 68.0673
Epoch 341/500, 

In [122]:
import numpy as np
import json

def generate_route(start_hold, num_steps=10, noise_level=0.02):
    model.eval()
    route = [start_hold]
    current = torch.tensor(start_hold, dtype=torch.float32).unsqueeze(0).unsqueeze(0)

    for _ in range(num_steps):
        next_hold = model(current).detach().numpy().flatten()

        # Add small random noise to prevent identical outputs
        next_hold[:2] += np.random.normal(0, noise_level, size=2)

        # Ensure values remain within valid range (0 to 1 after scaling)
        next_hold[:2] = np.clip(next_hold[:2], 0, 1)

        route.append(next_hold)
        current = torch.tensor(next_hold, dtype=torch.float32).unsqueeze(0).unsqueeze(0)

    # Convert numeric type back to label
    reverse_type_encoding = {0: "START", 1: "MIDDLE", 2: "FINISH", 3: "FEET-ONLY"}
    route_json = [
        {"x": float(hold[0]), "y": float(hold[1]), "type": reverse_type_encoding[int(round(hold[2]))]}
        for hold in route
    ]

    return json.dumps({"placements": route_json}, indent=4)

# Example: Generate a new climbing route
start_hold = np.array([0.5, 0.2, 0])  # Example normalized (x, y, type)
generated_route_json = generate_route(start_hold)

# Print generated JSON
print(generated_route_json)


{
    "placements": [
        {
            "x": 0.5,
            "y": 0.2,
            "type": "START"
        },
        {
            "x": 0.754388689994812,
            "y": 1.0,
            "type": "START"
        },
        {
            "x": 1.0,
            "y": 1.0,
            "type": "START"
        },
        {
            "x": 1.0,
            "y": 1.0,
            "type": "START"
        },
        {
            "x": 1.0,
            "y": 1.0,
            "type": "START"
        },
        {
            "x": 1.0,
            "y": 1.0,
            "type": "START"
        },
        {
            "x": 1.0,
            "y": 1.0,
            "type": "START"
        },
        {
            "x": 1.0,
            "y": 1.0,
            "type": "START"
        },
        {
            "x": 1.0,
            "y": 1.0,
            "type": "START"
        },
        {
            "x": 1.0,
            "y": 1.0,
            "type": "START"
        },
        {
            "x": 1.0,
  